In [ ]:
TEMP_DIR = "TEST"
BOT_TOKEN = "12345:6789"
ADMIN_CHAT_ID = 00000000
CHAT_ID = -00000000
YT_CHANNEL_OR_PLAYLIST_LINK = ""

from telebot import types
import telebot
import random
import time
import os
import re
from PIL import Image
from pathlib import Path
from functools import wraps
import uuid
import yt_dlp
import json
from telebot import types 


class DownloadedMedia:
    def __init__(self, file_name="", title="untitled", is_audio=False, duration=0, height=0, width=0):
        self.file_name = file_name
        self.title = title
        self.is_audio = is_audio
        self.duration = duration
        self.height = height
        self.width = width

    def to_json(self):
        return json.dumps(self.__dict__)

def reply_audio(audio_file: str, title: str, media_duration: int):
    thumbnail_filename = f'{audio_file}.jpg'
    thumbnail = Image.open(f'{audio_file}.webp').convert(
        'RGB').resize((840, 480))
    thumbnail.save(thumbnail_filename, 'jpeg')

    if Path(audio_file).is_file() and Path(thumbnail_filename).is_file():
        audio = open(audio_file, 'rb')
        bot.send_audio(chat_id=f'{CHAT_ID}',
                       audio=audio,
                       caption=title,
                       duration=media_duration,
                       title=title,
                       thumbnail=types.InputFile(thumbnail_filename)
         )
        audio.close()


def reply_video(file: str, title: str, media_duration: int):
    if Path(file).is_file():
        video = open(file, 'rb')
        bot.send_video(chat_id=f'{CHAT_ID}',
                       video=video,
                       caption=title,
                       duration=media_duration
         )
        video.close()


def download_audio_media(url: str):
    temp_file_name = str(uuid.uuid4())
    temp_file = os.path.join(TEMP_DIR, temp_file_name)
    os.makedirs(TEMP_DIR, exist_ok=True)
    info_file = f'{temp_file}.json'

    ydl_opts_audio = {
        "outtmpl": f"{TEMP_DIR}/%(title)s-%(duration)s",
        "noplaylist": False,
        "format": "bestaudio[filesize_approx<=50M][ext=m4a]/bestaudio[filesize<=50M][ext=m4a]/wa",
        'writethumbnail': True,
    }


    with  yt_dlp.YoutubeDL(ydl_opts_audio) as ydl_audio:
        info_audio = ydl_audio.extract_info(url, download=False)
        duration = info_audio.get("duration", 0)
        ydl_audio.download([url])
        is_audio = True

    return DownloadedMedia(temp_file, info_audio.get("title", "untitled"), is_audio, duration)


def download_video_edia(url: str):
    temp_file_name = str(uuid.uuid4())
    temp_file = os.path.join(TEMP_DIR, temp_file_name)
    os.makedirs(TEMP_DIR, exist_ok=True)
    info_file = f'{temp_file}.json'

    ydl_opts_audio = {
        "outtmpl": f"{TEMP_DIR}/%(title)s-%(duration)s",
        "noplaylist": False,
        "format": "best",
    }


    with  yt_dlp.YoutubeDL(ydl_opts_audio) as ydl_audio:
        info_audio = ydl_audio.extract_info(url, download=False)
        duration = info_audio.get("duration", 0)
        ydl_audio.download([url])
        is_audio = False

    return DownloadedMedia(temp_file, info_audio.get("title", "untitled"), is_audio, duration)

bot = telebot.TeleBot(BOT_TOKEN)

@bot.message_handler(content_types=["text"])
def echo(message):
    bot.send_message(ADMIN_CHAT_ID, "bot alive")

def send_files(folder_path):
    files = os.listdir(folder_path)
    sorted_files = sorted(files)

    for file_name in sorted_files:
        file_path = os.path.join(folder_path, file_name)

        sleep_duration = random.uniform(2, 5)
        time.sleep(sleep_duration)

        if os.path.isdir(file_path):
            continue  # Skip directories

        if not file_name.lower().endswith(".webp") and not file_name.lower().endswith(".jpg")  :
            duration = int(file_name.split('-')[-1])
            reply_audio(file_path, file_name.rsplit('-', 1)[0], duration)


if __name__ == "__main__":
    download_audio_media(YT_CHANNEL_OR_PLAYLIST_LINK)
    send_files(f'/content/{TEMP_DIR}')
    #bot.polling(none_stop=True, skip_pending=False)